# Quickstart : Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:

- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLFlow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform

In [12]:
# importing libraries
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import mlflow
from mlflow.models import infer_signature

In [13]:
# load dataset
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=';')

data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [14]:
# Split the data into train, validation and test split

train, test = train_test_split(data, test_size=0.25, random_state=17)

train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2330,7.5,0.38,0.29,12.7,0.050,25.0,209.0,0.9986,3.25,0.59,9.3,6
1194,6.2,0.37,0.22,8.3,0.025,36.0,216.0,0.9964,3.33,0.60,9.6,6
20,6.2,0.66,0.48,1.2,0.029,29.0,75.0,0.9892,3.33,0.39,12.8,8
1081,6.3,0.27,0.25,5.8,0.038,52.0,155.0,0.9950,3.28,0.38,9.4,6
1418,6.9,0.35,0.74,1.0,0.044,18.0,132.0,0.9920,3.13,0.55,10.2,5


In [15]:
train_x = train.drop(['quality'], axis=1).values
train_y = train[['quality']].values.ravel() # transforming here into single dimension array by using ravel()

test_x = test.drop(['quality'], axis=1).values
test_y = test['quality']

# Spliting train data into validation and train data
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.20, random_state=17)

# Storing the signature

signature = infer_signature(train_x, train_y)

train_x.shape, train_y.shape, valid_x.shape, valid_y.shape


((2938, 11), (2938,), (735, 11), (735,))

In [16]:
# ANN Model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential([
        keras.Input([train_x.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])

    # compile the model
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=params["lr"],momentum=params["momentum"]),
                  loss='mean_squared_error',
                  metrics=[keras.metrics.RootMeanSquaredError()]
        )
    
    # Train the ANN Model with lr and momentum params with MLFLOW tracking
    with mlflow.start_run(nested=True):
        
        model.fit(train_x, 
                  train_y,
                  validation_data=(valid_x, valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)

        eval_rmse = eval_result[1]

        # Log the params
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log the model        
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [17]:
# We need to create Objective function in order to track the parameters and results for each run

def objective(params):
    return train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )

In [18]:
# Initialize the hyperparameters
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0)
}

In [19]:
# start the mlflow experiment
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("wine-quality")

with mlflow.start_run():
    # conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss and model    
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])

    # Log the model        
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # print the best parameters and corresponding loss
    print(f"Best Parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 31s 692ms/step - loss: 38.2921 - root_mean_squared_error: 6.1279
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - loss: 37.5544 - root_mean_squared_error: 6.1279
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37.1950 - root_mean_squared_error: 6.0731
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.8872 - root_mean_squared_error: 6.0731
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.5692 - root_mean_squared_error: 6.0140
 6/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.1797 - root_mean_squared_error: 6.0140
 8/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 35.4768 - root_mean_squared_error: 5.9547
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 35.8175 - root_mean_squared_error: 5.9547   
 9/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 35.1208 - root_mean_squared_error: 5.9242   
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.3171 - root_mean_squared_error: 5.4700  
26/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - l

2024/11/02 11:26:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run caring-conch-881 at: http://127.0.0.1:5000/#/experiments/648921611001114077/runs/8169509baac74cb88814cd1e222ded27.

2024/11/02 11:26:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/648921611001114077.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 36s 802ms/step - loss: 29.5119 - root_mean_squared_error: 5.4325
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.3920 - root_mean_squared_error: 5.4214
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 29.5713 - root_mean_squared_error: 5.4379
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 29.7230 - root_mean_squared_error: 5.4518
 6/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.9074 - root_mean_squared_error: 5.4687
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 29.8256 - root_mean_squared_error: 5.4723
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.9472 - root_mean_squared_error: 5.4723
 8/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 29.9559 - root_mean_squared_error: 5.4736
 9/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 29.9613 - root_mean_squared_error: 5.4736
10/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 29.9751 - root_mean_squared_error: 5.4741   
30/46 ━━━━━━━━━━━━━━━━━

2024/11/02 11:26:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run honorable-hare-800 at: http://127.0.0.1:5000/#/experiments/648921611001114077/runs/e2d79fb2da4a4ffc8e853e1ee5e7fba5.

2024/11/02 11:26:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/648921611001114077.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 30s 686ms/step - loss: 36.6395 - root_mean_squared_error: 5.9348
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 35.2355 - root_mean_squared_error: 5.9348
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.9523 - root_mean_squared_error: 5.9112
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.6653 - root_mean_squared_error: 5.8741
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.5134 - root_mean_squared_error: 5.8741
 6/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.4536 - root_mean_squared_error: 5.8670
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.4279 - root_mean_squared_error: 5.8670
21/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.2987 - root_mean_squared_error: 5.8548
22/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.2804 - root_mean_squared_error: 5.8548
23/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.2075 - root_mean_squared_error: 5.8485
24/46 ━━━━━━━━━━━━━━━━━━━━

...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 31s 690ms/step - loss: 37.3916 - root_mean_squared_error: 6.1149
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.7952 - root_mean_squared_error: 5.8946
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.7067 - root_mean_squared_error: 5.5258
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32.6102 - root_mean_squared_error: 5.7011
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28.9740 - root_mean_squared_error: 5.3596
 6/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 27.3432 - root_mean_squared_error: 5.1964
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 25.8802 - root_mean_squared_error: 5.0449
 8/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 24.5917 - root_mean_squared_error: 4.7806
 9/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 23.4348 - root_mean_squared_error: 4.7806
10/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 22.3962 - root_mean_squared_error: 4.6638
28/46 ━━━━━━━━━━━━━━━━━━━━

2024/11/02 11:27:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run bouncy-mink-379 at: http://127.0.0.1:5000/#/experiments/648921611001114077/runs/1505de8e064d466cacebd2e442f25c56.

2024/11/02 11:27:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/648921611001114077.



100%|██████████| 4/4 [00:56<00:00, 14.18s/trial, best loss: 0.8383307456970215]


2024/11/02 11:27:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run melodic-sheep-314 at: http://127.0.0.1:5000/#/experiments/648921611001114077/runs/9878417a13c14327b45abedb80dee0e5.
2024/11/02 11:27:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/648921611001114077.


Best Parameters: {'lr': 0.0156448466399016, 'momentum': 0.09155564956195505}
Best eval rmse: 0.8383307456970215


So the result by best model,

Best Parameters: {'lr': 0.0156448466399016, 'momentum': 0.09155564956195505}


Best eval rmse: 0.8383307456970215

In [20]:
# Inferencing

from mlflow.models import validate_serving_input

model_uri = 'runs:/9878417a13c14327b45abedb80dee0e5/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)


/Users/starlord/Desktop/Work/Learn/MLOps/started-with-mlflow/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[5.994581 ],
       [5.692972 ],
       [6.1674457],
       ...,
       [5.5002685],
       [6.922455 ],
       [5.639419 ]], dtype=float32)

In [21]:
# We can do inferencing by loading the model as well,

import mlflow
logged_model = 'runs:/9878417a13c14327b45abedb80dee0e5/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[5.994581 ],
       [5.692972 ],
       [6.1674457],
       ...,
       [5.5002685],
       [6.922455 ],
       [5.639419 ]], dtype=float32)